In [1]:
from __future__ import division, print_function, absolute_import

from skimage import color, io
from scipy.misc import imresize
import numpy as np
from sklearn.cross_validation import train_test_split
import os
from glob import glob

import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
from tflearn.metrics import Accuracy

/opt/anaconda/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
files_path = '/home/hfan/projects/Heidi/train_ant/'

yes_files_path = os.path.join(files_path, 'yes_*.png')
no_files_path = os.path.join(files_path, 'no_*.png')

yes_files = sorted(glob(yes_files_path))
no_files = sorted(glob(no_files_path))

n_files = len(yes_files) + len(no_files)
print(n_files)


36725


### Getting a balanced and managable dataset

In [3]:
import random, os 

each_set = 3672 *2

yes_idx = random.sample(range(len(yes_files)),each_set)
no_idx = random.sample(range(len(no_files)), each_set)


In [4]:
size_image = 64
allX = np.zeros((each_set * 2, size_image, size_image, 3), dtype='float64')
ally = np.zeros(each_set * 2)
count = 0
for idx in yes_idx:
    try:
        img = io.imread(yes_files[idx])
        new_img = imresize(img, (size_image, size_image, 3))
        allX[count] = np.array(new_img)
        ally[count] = 0
        count += 1
    except:
        continue
        
for idx in no_idx:
    try:
        img = io.imread(no_files[idx])
        new_img = imresize(img, (size_image, size_image, 3))
        allX[count] = np.array(new_img)
        ally[count] = 1
        count += 1
    except:
        continue

In [5]:
# test-train split   
X, X_test, Y, Y_test = train_test_split(allX, ally, test_size=0.1, random_state=42)

# encode the Ys
Y = to_categorical(Y, 2)
Y_test = to_categorical(Y_test, 2)

In [5]:
# normalisation of images
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Create extra synthetic training data by flipping & rotating images
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)

In [6]:
# Input is a 32x32 image with 3 color channels (red, green and blue)
network = input_data(shape=[None, 64, 64, 3],
                     data_preprocessing=img_prep, 
                     data_augmentation=img_aug)

# 1: Convolution layer with 32 filters, each 3x3x3
conv_1 = conv_2d(network, 32, 3, activation='relu', name='conv_1')

# 2: Max pooling layer
network = max_pool_2d(conv_1, 2)

# 3: Convolution layer with 64 filters
conv_2 = conv_2d(network, 64, 3, activation='relu', name='conv_2')

# 4: Convolution layer with 64 filters
conv_3 = conv_2d(conv_2, 64, 3, activation='relu', name='conv_3')

# 5: Max pooling layer
network = max_pool_2d(conv_3, 2)

# 6: Fully-connected 512 node layer
network = fully_connected(network, 512, activation='relu')

# 7: Dropout layer to combat overfitting
network = dropout(network, 0.5)

# 8: Fully-connected layer with two outputs
network = fully_connected(network, 2, activation='softmax')


In [7]:
# Configure how the network will be trained
acc = Accuracy(name="Accuracy")
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.0005, metric=acc)

# Wrap the network in a model object
model = tflearn.DNN(network, checkpoint_path='ant_7344.tflearn', max_checkpoints = 3,
                    tensorboard_verbose = 3, tensorboard_dir='tmp/tflearn_logs/')


In [9]:
###################################
# Train model for 10 epochs
###################################
model.fit(X, Y, validation_set=(X_test, Y_test), batch_size=100,
      n_epoch=100, run_id='ant_14688', show_metric=True)

model.save('model_sub_ant_20170811_batch100_epoch100_14688pics.tflearn')

Training Step: 13299  | total loss: 0.09382 | time: 158.983s
| Adam | epoch: 100 | loss: 0.09382 - Accuracy: 0.9625 -- iter: 13200/13219
Training Step: 13300  | total loss: 0.09898 | time: 164.226s
| Adam | epoch: 100 | loss: 0.09898 - Accuracy: 0.9632 | val_loss: 0.48460 - val_acc: 0.8856 -- iter: 13219/13219
--


### Prediction

In [2]:
yes_file_path = '/home/hfan/projects/Heidi/test_ant/yes*.png'
no_file_path = '/home/hfan/projects/Heidi/test_ant/no*.png'

yes_file_prd = glob(yes_file_path)
no_file_prd = glob(no_file_path)

size_image = 64

predictYes = np.zeros((len(yes_file_prd), size_image, size_image, 3), dtype='float64')
predictNo = np.zeros((len(no_file_prd), size_image, size_image, 3), dtype='float64')

count = 0

for f in yes_file_prd:
    try:
        img = io.imread(f)
        new_img = imresize(img, (size_image, size_image, 3))
        predictYes[count] = np.array(new_img)
        count += 1
    except:
        continue
        
count = 0
for f in no_file_prd:
    try:
        img = io.imread(f)
        new_img = imresize(img, (size_image, size_image, 3))
        predictNo[count] = np.array(new_img)
        count += 1
    except:
        continue


In [8]:
model.load('model_sub_ant_20170811_batch100_epoch100_14688pics.tflearn')

INFO:tensorflow:Restoring parameters from /home/hfan/projects/Heidi/model_sub_ant_20170811_batch100_epoch100_14688pics.tflearn


In [36]:
results = model.predict(predictNo[2800:])

In [37]:
def accuracy(results, truth = 0):
    right = 0
    wrong = 0
    idx = 0
    wrong_idx = []
    for result in results:
        if np.argmax(result) == truth:
            right += 1
        else:
            wrong += 1
            wrong_idx.append(idx)
        idx += 1
    print(right/(right + wrong))
    return wrong_idx
    
wrong_idx = accuracy(results, 1)    

0.896551724137931


In [17]:
print(len(predictYes),len(predictNo))

1250 2829


In [46]:
for idx in wrong_idx:
    print(no_files[idx])

/home/hfan/projects/Heidi/test_ant/no_09MAY2016_ants_11_488.png
/home/hfan/projects/Heidi/test_ant/no_09MAY2016_ants_64_247.png
/home/hfan/projects/Heidi/test_ant/no_09MAY2016_ants_39_147.png
/home/hfan/projects/Heidi/test_ant/no_09MAY2016_ants_49_220.png
/home/hfan/projects/Heidi/test_ant/no_09MAY2016_ants_19_102.png
/home/hfan/projects/Heidi/test_ant/no_09MAY2016_ants_21_496.png
/home/hfan/projects/Heidi/test_ant/no_09MAY2016_ants_29_21.png
/home/hfan/projects/Heidi/test_ant/no_09MAY2016_ants_65_83.png
/home/hfan/projects/Heidi/test_ant/no_09MAY2016_ants_10_509.png
/home/hfan/projects/Heidi/test_ant/no_11MAY2016_ants_68_189.png
/home/hfan/projects/Heidi/test_ant/no_09MAY2016_ants_20_244.png
/home/hfan/projects/Heidi/test_ant/no_09MAY2016_ants_40_84.png
/home/hfan/projects/Heidi/test_ant/no_09MAY2016_ants_49_365.png
/home/hfan/projects/Heidi/test_ant/no_09MAY2016_ants_39_406.png
/home/hfan/projects/Heidi/test_ant/no_09MAY2016_ants_2_254.png
/home/hfan/projects/Heidi/test_ant/no_09MAY2

In [23]:
print(np.argmax(results[0]))

0


In [24]:
print(np.amax(results[0]))

0.529458
